In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
load_url = "https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=040&bs=040&ta=20&sc=20214&sngz=&po1=25&pc=50"

res = requests.get(load_url)
res.encoding = "utf-8"
soup = BeautifulSoup(res.text, "html.parser")

In [7]:
cassetitems = soup.find_all("div", class_="cassetteitem")

for i in range(len(cassetitems)):
    
    # 上の部分（建物情報）
    details = cassetitems[i].find_all("div", class_="cassetteitem-detail")
    
    for ii in range(len(details)):
        name = details[ii].find("div", class_="cassetteitem_content-title").text
        address = details[ii].find("li", class_="cassetteitem_detail-col1").text
        _stations = details[ii].find_all("div", class_="cassetteitem_detail-text")
        station1 = _stations[0].text
        station2 = _stations[1].text
        station3 = _stations[2].text
        _col3 = details[ii].find_all("li", class_="cassetteitem_detail-col3")
        building_age = _col3[0].find_all("div")[0].text
        number_of_floors = _col3[0].find_all("div")[1].text
        
        # 下の部分（部屋の情報）
        items = cassetitems[i].find("div", class_="cassetteitem-item")
        tbodys = items.find_all("tbody")
        for iii in range(len(tbodys)):
            _tds = tbodys[iii].find_all("td")
            floor = _tds[2].text.split('\n')[-1].replace('\t','')
            rent = _tds[3].find("span", class_="cassetteitem_other-emphasis ui-text--bold").text
            maintenance_fee = _tds[3].find("span", class_="cassetteitem_price cassetteitem_price--administration").text
            deposit = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--deposit").text
            gratuity = _tds[4].find("span", class_="cassetteitem_price cassetteitem_price--gratuity").text
            layout = _tds[5].find("span", class_="cassetteitem_madori").text
            area = _tds[5].find("span", class_="cassetteitem_menseki").text
            

            print(name)
            print(address)
            print(station1)
            print(station2)
            print(station3)
            print(building_age)
            print(number_of_floors)
            print(floor)
            print(rent)
            print(maintenance_fee)
            print(deposit)
            print(gratuity)
            print(layout)
            print(area)
            
            

ヒルサイドケンタッキー
長野県茅野市ちの横内
ＪＲ中央線/茅野駅 歩13分
ＪＲ中央線/上諏訪駅 歩81分

築25年
2階建
1階
5.3万円
4300円
-
-
2DK
44.34m2
リバーサイドスクウェアＡ
長野県茅野市豊平
ＪＲ中央線/茅野駅 歩54分


築23年
2階建
1階
4.4万円
5300円
-
-
2DK
40.04m2
ヒーローマンション本町東
長野県茅野市本町西
ＪＲ中央線/茅野駅 歩30分


築23年
3階建
3階
3.8万円
1800円
3.8万円
-
1K
24.96m2
エスポワール２１Ｃ
長野県茅野市塚原１
ＪＲ中央線/茅野駅 歩9分


築24年
2階建
1階
5.35万円
4700円
-
-
2DK
44.34m2
ＪＲ中央線 茅野駅 2階建 築15年
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩17分


築15年
2階建
1階
5.8万円
5100円
-
8.7万円
1LDK
42.63m2
ＪＲ中央線 茅野駅 2階建 築15年
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩17分


築15年
2階建
2階
6.5万円
5100円
-
9.75万円
2LDK
54.67m2
カーサ・フィオーレＢ
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩15分


築15年
2階建
1階
5.8万円
5100円
-
8.7万円
1LDK
42.63m2
カーサ・フィオーレＢ
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩15分


築15年
2階建
2階
6.5万円
5100円
-
9.75万円
2LDK
54.67m2
カーサ・フィオーレＢ
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩15分


築15年
2階建
1階
5.8万円
51000円
-
8.7万円
1LDK
42.63m2
サンセール
長野県茅野市玉川
ＪＲ中央線/茅野駅 バス18分 (バス停)田道 歩4分


築16年
2階建
2階
6.4万円
5500円
-
9.6万円
2LDK
58.86m2
レオパレスエスターテ
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩16分


築15年
2階建
1階
4.4万円
5000円
-
4.4万円
1K
23.18m2
OBER CHINO
長野県茅野市ちの
ＪＲ中央線/茅野駅 歩6分
ＪＲ中央線/上諏訪駅 歩83分
ＪＲ中央線/青柳駅 歩99分
築1

In [6]:
print(name)
print(address)
print(station1)
print(station2)
print(station3)
print(building_age)
print(number_of_floors)
print(floor)
print(rent)
print(maintenance_fee)
print(deposit)
print(gratuity)
print(layout)
print(area)

ブルック　ウエスト　II
長野県茅野市宮川
ＪＲ中央線/茅野駅 歩16分


築11年
2階建
1階
6.5万円
3300円
-
9.75万円
1LDK
46.24m2
